In [1]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def create_stack(ystack, axis, savetodir, downsample_factor, scalebar_microns = 100):
    '''
    Creates a stack of images along the specified axis, using a source y-stack of images.
    Args:
        ystack: A list of images (np array) to stack.
        axis: The axis along which to stack the images. Can be 'x', 'y', or 'z'.
        downsample_factor: Factor by which to downsample the image in all three dimensions. Default is 1 (no downsampling).
    '''
    # Downsample every image in the source stack
    ystack = [cv2.resize(img, (img.shape[1]//downsample_factor, img.shape[0]//downsample_factor)) for img in ystack]

    # Delete every downsample_factor image in the source stack
    ystack = [img for i, img in enumerate(ystack) if i % downsample_factor == 0]

    if axis == 'x':
        print('Creating x stack...')
        stack = [np.zeros((len(ystack), ystack[0].shape[0], 3), dtype=np.uint8) for _ in range(ystack[0].shape[1])]
        for x in tqdm(range(ystack[0].shape[1])):
            edges = [img[:, x:x+1, :].reshape(1, img.shape[0], 3) for img in ystack]
            for i, edge in enumerate(edges):
                stack[x][i:i+1, :, :] = edge
            # cv2.putText(stack[x], f'x = {x*downsample_factor}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    elif axis == 'z':
        print('Creating z stack...')
        stack = [np.zeros((len(ystack), ystack[0].shape[1], 3), dtype=np.uint8) for _ in range(ystack[0].shape[0])]
        for z in tqdm(range(ystack[0].shape[0])):
            edges = [img[z:z+1, :, :].reshape(1, img.shape[1], 3) for img in ystack]
            for i, edge in enumerate(edges):
                stack[z][i:i+1, :, :] = edge
            # cv2.putText(stack[z], f'z = {z*downsample_factor}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    elif axis == 'y':
        print('Creating y stack...')
        stack = []
        for i, img in tqdm(enumerate(ystack), total = len(ystack)):
            # cv2.putText(img, f'y = {i*downsample_factor}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            # Dim the scale bar background
            img[-90:-15, -160:-10, :] = img[-90:-15, -160:-10, :] * 0.5
            # Add a scale bar to the bottom right corner of the image
            img[-80:-70, -135:-35, :] = 255
            # Annotate scale bar with length
            cv2.putText(img, f'{scalebar_microns} um', (img.shape[1]-150, img.shape[0]-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            stack.append(img)
    else:
        raise Exception('axis must be one of x, y, or z')

    os.makedirs(os.path.join(savetodir, axis), exist_ok = True)
    for i, img in enumerate(stack):
        cv2.imwrite(os.path.join(savetodir, axis, f'{i}.png'), img)

In [2]:
import json
with open('metadata.json', 'r') as f:
    metadata = json.load(f)

new_metadata = []
for item in metadata:
    ID = item['id']

    new_item = {
                'id': ID,
                'metadata': item['metadata'],
                'dimensions': {}
                }

    for mode in ['raw', 'heatmap']:
        SOURCE_DIR = f'data/{ID}/{mode}/orig'
        DEST_DIR = f'data_hi_res/{ID}/{mode}'
        DOWNSAMPLE_FACTOR = 1
        if item['metadata']['type'] == 'CT':
            SCALEBAR_MICRONS = 400
        elif item['metadata']['type'] == 'OTLS':
            SCALEBAR_MICRONS = 100
        else:
            raise Exception(f'Invalid type: {item["metadata"]["type"]} not in ["CT", "OTLS"]')

        # Load images
        imagepaths = sorted(os.listdir(SOURCE_DIR), key = lambda x: int(x.split('_')[-1].split('.')[0]))  # Sort paths by index
        images = [cv2.imread(os.path.join(SOURCE_DIR, f)) for f in imagepaths]
        print(f'Creating xyz stacks from {len(images)} images in {SOURCE_DIR}...')

        # Create stacks
        create_stack(images, 'x', DEST_DIR, DOWNSAMPLE_FACTOR)
        create_stack(images, 'z', DEST_DIR, DOWNSAMPLE_FACTOR)
        create_stack(images, 'y', DEST_DIR, DOWNSAMPLE_FACTOR, SCALEBAR_MICRONS)

        # Print dimensions
        print(f'Width: {len(os.listdir(os.path.join(DEST_DIR, "x")))}')
        print(f'Depth: {len(os.listdir(os.path.join(DEST_DIR, "z")))}')
        print(f'Height: {len(os.listdir(os.path.join(DEST_DIR, "y")))}')

    # Add metadata
    new_item['dimensions']['x'] = len(os.listdir(os.path.join(DEST_DIR, "x")))
    new_item['dimensions']['z'] = len(os.listdir(os.path.join(DEST_DIR, "z")))
    new_item['dimensions']['y'] = len(os.listdir(os.path.join(DEST_DIR, "y")))
    new_metadata.append(new_item)

with open('metadata_hi_res.json', 'w') as f:
    json.dump(new_metadata, f, indent = 4)

Creating xyz stacks from 351 images in data/CT_31263/raw/orig...
Creating x stack...


100%|██████████| 2200/2200 [01:16<00:00, 28.82it/s]


Creating z stack...


100%|██████████| 1300/1300 [00:13<00:00, 93.57it/s]


Creating y stack...


...

Creating z stack...


100%|██████████| 5900/5900 [00:02<00:00, 2293.04it/s]


Creating y stack...


100%|██████████| 201/201 [00:00<00:00, 2753.40it/s]


Width: 350
Depth: 5900
Height: 201
